In [ ]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import pandas as pd
import matplotlib.pyplot as plt

# Import the CRUD module
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Initialize the AnimalShelter instance with username and password
username = "aacuser"
password = "SNHU1234"
db = AnimalShelter()

# Read the data from the MongoDB using the read method (retrieve all data)
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ will return the '_id' column, which is not compatible with Dash DataTable,
# so we remove it from the DataFrame
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################

# Initialize the Dash app
app = JupyterDash(__name__)

# Grazioso Salvare logo integration
image_filename = 'Grazioso_Salvare_Logo.png'  # Path to logo file
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Layout of the Dashboard
app.layout = html.Div([
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard'))),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '200px'}),
    html.Hr(),

    # Filter options - dropdown for Rescue Type
    html.Label('Select Rescue Type:'),
    dcc.Dropdown(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',  # Default value
    ),
    html.Hr(),

    # Interactive data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,  # Display 10 rows per page
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data_conditional=[],
    ),

    html.Br(),
    html.Hr(),

    # This sets up the dashboard so that your chart and geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(
                id='graph-id',
                className='col s12 m6',
            ),
            html.Div(
                id='map-id',
                className='col s12 m6',
            )
        ]
    )
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback to filter the data table based on Rescue Type selection
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Water Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["Labrador Retriever Mix", "Golden Retriever"]}}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"animal_type": "Dog", "breed": {"$in": ["German Shepherd", "Alaskan Malamute"]}}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"animal_type": "Dog", "breed": {"$in": ["Bloodhound", "Coonhound"]}}
    else:  # Reset case
        query = {}

    # Fetch the filtered data from MongoDB
    df_filtered = pd.DataFrame.from_records(db.read(query))
    if '_id' in df_filtered.columns:
        df_filtered.drop(columns=['_id'], inplace=True)
    return df_filtered.to_dict('records')

# Callback to update the pie chart based on the filtered data
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None:
        dff = df
    else:
        dff = pd.DataFrame(viewData)

    # Generate pie chart for breed distribution
    pie_chart = px.pie(dff, names='breed', title='Preferred Breeds for Rescue')
    return [
        dcc.Graph(
            figure=pie_chart
        )
    ]

# Callback to highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Callback to update the geo-location chart for the selected data entry
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_map(viewData):
    if viewData is None:
        dff = df
    else:
        dff = pd.DataFrame(viewData)

    # If no data is available, return an empty map
    if dff.empty:
        return dl.Map(center=[30.27, -97.74], zoom=10, children=[dl.TileLayer()])

    # Assume Austin, Texas is the location (for illustration purposes)
    markers = [dl.Marker(position=[30.27, -97.74], children=[dl.Tooltip(dff['breed'].iloc[0])])]
    return dl.Map(center=[30.27, -97.74], zoom=10, children=[dl.TileLayer(), *markers])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)